<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 3

### Regression and Classification with the Ames Housing Data

---

You have just joined a new "full stack" real estate company in Ames, Iowa. The strategy of the firm is two-fold:
- Own the entire process from the purchase of the land all the way to sale of the house, and anything in between.
- Use statistical analysis to optimize investment and maximize return.

The company is still small, and though investment is substantial the short-term goals of the company are more oriented towards purchasing existing houses and flipping them as opposed to constructing entirely new houses. That being said, the company has access to a large construction workforce operating at rock-bottom prices.

This project uses the [Ames housing data recently made available on kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).

In [104]:
# Import packages
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from __future__ import division

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Estimating the value of homes from fixed characteristics.

---

Your superiors have outlined this year's strategy for the company:
1. Develop an algorithm to reliably estimate the value of residential houses based on *fixed* characteristics.
2. Identify characteristics of houses that the company can cost-effectively change/renovate with their construction team.
3. Evaluate the mean dollar value of different renovations.

Then we can use that to buy houses that are likely to sell for more than the cost of the purchase plus renovations.

Your first job is to tackle #1. You have a dataset of housing sale data with a huge amount of features identifying different aspects of the house. The full description of the data features can be found in a separate file:

    housing.csv
    data_description.txt
    
You need to build a reliable estimator for the price of the house given characteristics of the house that cannot be renovated. Some examples include:
- The neighborhood
- Square feet
- Bedrooms, bathrooms
- Basement and garage space

and many more. 

Some examples of things that **ARE renovate-able:**
- Roof and exterior features
- "Quality" metrics, such as kitchen quality
- "Condition" metrics, such as condition of garage
- Heating and electrical components

and generally anything you deem can be modified without having to undergo major construction on the house.

---

**Your goals:**
1. Perform any cleaning, feature engineering, and EDA you deem necessary.
- Be sure to remove any houses that are not residential from the dataset.
- Identify **fixed** features that can predict price.
- Train a model on pre-2010 data and evaluate its performance on the 2010 houses.
- Characterize your model. How well does it perform? What are the best estimates of price?

> **Note:** The EDA and feature engineering component to this project is not trivial! Be sure to always think critically and creatively. Justify your actions! Use the data description file!

# Initial Exploration of Data

In [2]:
# Load the data
house = pd.read_csv('./housing.csv')

# Print head of house dataframe
house.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# Print tail of dataframe
house.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500


In [131]:
# Print column headers of house
house.columns.values

array(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir',
       'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'Pav

# Data Cleaning:
The data will be cleaned by:
* Removing duplicates
* Checking for null values
* Conversion of compatible columns to non-object types if possible
        

In [5]:
# Initial check of shape before manipulation
house.shape

(1460, 81)

In [6]:
# Drop duplicates found within dataframe
house.drop_duplicates()

# Check shape of dataframe to determine if changes occurred
house.shape

(1460, 81)

## Duplicates
It appears that there are no duplicates within the dataset. This is seen by calling .shape on the dataset before and after removing duplicates to check for changes

# Datatypes
* Using house.info() and the text file describing the dataset, we are able to determine which columns are considered categorical, and which ones contain continous data.

In [7]:
house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

## Continous Data
* MasVnrArea: Masonry veneer area in square feet
* LotFrontage: Linear feet of street connected to property
* LotArea: Lot size in square feet
* YearBuilt: Original construction date
* YearRemodAdd: Remodel date (same as construction date if no remodeling or additions)
* BsmtFinSF1: Type 1 finished square feet
* BsmtFinSF2: Type 2 finished square feet
* BsmtUnfSF: Unfinished square feet of basement area
* TotalBsmtSF: Total square feet of basement area
* 1stFlrSF: First Floor square feet
* 2ndFlrSF: Second floor square feet
* LowQualFinSF: Low quality finished square feet (all floors)
* GrLivArea: Above grade (ground) living area square feet
* GarageYrBlt: Year garage was built
* GarageArea: Size of garage in square feet
* WoodDeckSF: Wood deck area in square feet
* OpenPorchSF: Open porch area in square feet
* EnclosedPorch: Enclosed porch area in square feet
* 3SsnPorch: Three season porch area in square feet
* ScreenPorch: Screen porch area in square feet
* PoolArea: Pool area in square feet
* MiscVal: (Dollar)Value of miscellaneous feature
* YrSold: Year Sold (YYYY)


## Discrete (potentially)

* BsmtFullBath: Basement full bathrooms
* BsmtHalfBath: Basement half bathrooms
* FullBath: Full bathrooms above grade
* HalfBath: Half baths above grade
* Bedroom: Bedrooms above grade (does NOT include basement bedrooms)
* Kitchen: Kitchens above grade
* TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
* Fireplaces: Number of fireplaces
* GarageCars: Size of garage in car capacity
* MoSold: Month Sold (MM)


## Steps
* Attempt to convert all above points into non-object types. Other columns tend to hold categorical data, sometimes in the form of code strings (eg. two letter notation for MSzoning) or 1-10 ranking metrics.
* Those considered optional qualities of a house (such as Pool or fireplace) null values, are tolerated.


In [8]:
# Check for null values in each column
house.isnull().sum().sort_values(ascending=False)

PoolQC           1453
MiscFeature      1406
Alley            1369
Fence            1179
FireplaceQu       690
LotFrontage       259
GarageCond         81
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageQual         81
BsmtExposure       38
BsmtFinType2       38
BsmtFinType1       37
BsmtCond           37
BsmtQual           37
MasVnrArea          8
MasVnrType          8
Electrical          1
Utilities           0
YearRemodAdd        0
MSSubClass          0
Foundation          0
ExterCond           0
ExterQual           0
Exterior2nd         0
Exterior1st         0
RoofMatl            0
RoofStyle           0
YearBuilt           0
                 ... 
GarageArea          0
PavedDrive          0
WoodDeckSF          0
OpenPorchSF         0
3SsnPorch           0
BsmtUnfSF           0
ScreenPorch         0
PoolArea            0
MiscVal             0
MoSold              0
YrSold              0
SaleType            0
Functional          0
TotRmsAbvGrd        0
KitchenQua

Upon closer inspection of the columns' null values, it is found that the null values are within columns that have reasonable reasons to be null. The important columns of MSZoning (used for filtering residential properties) and SalePrice (used as our target) contain no null values.

## Potential Dummy Variables
* MSSubClass: 16 dummies
* MSZoning: 8 Dummies
* Street: 4 Dummies
* Alley: 3 dummies
* LotShape: 4 dummies
* LandContour: 4 dummies
* Utilities: 4 dummies
* LotConfig: 4 dummies
* LandSlope: 3 dummies
* Neighborhood: 25 dummies
* Condition1: 9 dummies
* BldgType: 5 dummies
* HouseStyle: 8 dummies
* RoofStyle: 6 dummies
* RoofMatl: 8 dummies
* Exterior!st: 17 dummies
* MasVnrType: 5 dummies
* Foundation: 6 dummies
* BsmtExposure: 5 dummies
* Heating: 6 dummies
* CentralAir: Replace in column with binary
* Electrical: 5 dummies
* Functional: 8 dummmies
* GarageType: 7 dummies
* PavedDrive: 3 dummies
* MiscFeature: 6 dummies
* SaleType: 10 dummies
* SaleCondition: 6 dummies

## Conversion from categorical to numeric?
* ExterQual: Replace with 1-5?
* ExterCond: Replace 1-5?
* BsmtQual: Replace with values? (maybe)
* BsmtCond: Replace ith values 1-5
* BsmtFinType1: Replace with 7 levels
* HeatingQC: Replace with 5 levels
* KitchenQual: Replace with 1-5
* FireplaceQu: Replace with 0-5
* GarageFinish: 0-3
* GarageQual: 0-5
* GarageCond: 0-5
* PoolQC: 0-4
* Fence: 0-4

# Dummy Variable Creation and Concatenation

In [9]:
# Create dummy df of MSSubClass
Subclassdum = pd.get_dummies(house['MSSubClass'])

# Rename columns 
Subclassdum.columns = ['MSSub_20','MSSub_30','MSSub_40','MSSub_45','MSSub_50','MSSub_60','MSSub_70','MSSub_75',
                      'MSSub_80','MSSub_85','MSSub_90','MSSub_120','MSSub_160','MSSub_180','MSSub_190']

# Concatenate dummy variables into main df: house
house = pd.concat([house, Subclassdum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MSSub_60,MSSub_70,MSSub_75,MSSub_80,MSSub_85,MSSub_90,MSSub_120,MSSub_160,MSSub_180,MSSub_190
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,1,0,0,0,0,0,0,0,0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,0,0,0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,1,0,0,0,0,0,0,0,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,1,0,0,0,0,0,0,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,1,0,0,0,0,0,0,0,0,0


In [10]:
# Create dummy df of MSZoning
Zoneclassdum = pd.get_dummies(house['MSZoning'])

# Rename columns
Zoneclassdum.columns = ['MSZon_Commercial','MSZon_Resi_FloatVillage','MSZon_Resi_HighDensity','MSZon_Resi_LowDensity',
                        'MSZon_Resi_MedDensity']

# Concatenate dummy variables into main df: house
house = pd.concat([house, Zoneclassdum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MSSub_90,MSSub_120,MSSub_160,MSSub_180,MSSub_190,MSZon_Commercial,MSZon_Resi_FloatVillage,MSZon_Resi_HighDensity,MSZon_Resi_LowDensity,MSZon_Resi_MedDensity
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,0,0,1,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,0,0,1,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,0,1,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,0,1,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,0,1,0


In [11]:
# Create dummies
streetdum = pd.get_dummies(house['Street'])

# Rename columns
streetdum.columns = ['Street_Gravel', 'Street_Paved']

# Concatenate dummy variables into main df: house
house = pd.concat([house, streetdum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MSSub_160,MSSub_180,MSSub_190,MSZon_Commercial,MSZon_Resi_FloatVillage,MSZon_Resi_HighDensity,MSZon_Resi_LowDensity,MSZon_Resi_MedDensity,Street_Gravel,Street_Paved
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,1,0,0,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,1,0,0,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,1,0,0,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,1,0,0,1
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,1,0,0,1


In [12]:
# Create dummies
alleydum = pd.get_dummies(house['Alley'])

# Rename columns
alleydum.columns = ['Alley_Gravel', 'Alley_Paved']

# Concatenate dummy variables into main df: house
house = pd.concat([house, alleydum], axis=1, join='outer')
house.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MSSub_190,MSZon_Commercial,MSZon_Resi_FloatVillage,MSZon_Resi_HighDensity,MSZon_Resi_LowDensity,MSZon_Resi_MedDensity,Street_Gravel,Street_Paved,Alley_Gravel,Alley_Paved
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,1,0,0,1,0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,1,0,0,1,0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,1,0,0,1,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,1,0,0,1,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,1,0,0,1,0,0


In [13]:
# Create dummies
lotshapedum = pd.get_dummies(house['LotShape'])

# Rename columns
lotshapedum.columns = ['LotSha_SlightIrregular','LotSha_ModIrregular','LotSha_Irregular',
                       'LotSha_Regular']

# Concatenate dummy variables into main df: house
house = pd.concat([house, lotshapedum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MSZon_Resi_LowDensity,MSZon_Resi_MedDensity,Street_Gravel,Street_Paved,Alley_Gravel,Alley_Paved,LotSha_SlightIrregular,LotSha_ModIrregular,LotSha_Irregular,LotSha_Regular
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,1,0,0,1,0,0,0,0,0,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,1,0,0,1,0,0,0,0,0,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,1,0,0,1,0,0,1,0,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,1,0,0,1,0,0,1,0,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,1,0,0,1,0,0,1,0,0,0


In [14]:
# Create dummies
contourdum = pd.get_dummies(house['LandContour'])

# Rename columns
contourdum.columns = ['LContour_Banked','LContour_Hillside','LContour_Depression','LContour_Flat']

# Concatenate dummy variables into main df: house
house = pd.concat([house, contourdum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Alley_Gravel,Alley_Paved,LotSha_SlightIrregular,LotSha_ModIrregular,LotSha_Irregular,LotSha_Regular,LContour_Banked,LContour_Hillside,LContour_Depression,LContour_Flat
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,1,0,0,0,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,1,0,0,0,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,1,0,0,0,0,0,0,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,1,0,0,0,0,0,0,1
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,1,0,0,0,0,0,0,1


In [15]:
# Create dummies
hooddum = pd.get_dummies(house['Neighborhood'])

# Rename columns
for i in hooddum.columns:
    hooddum.rename(columns={i: 'Hood_'+str(i)}, inplace=True)

# Concatenate dummy variables into main df: house
house = pd.concat([house, hooddum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Hood_NoRidge,Hood_NridgHt,Hood_OldTown,Hood_SWISU,Hood_Sawyer,Hood_SawyerW,Hood_Somerst,Hood_StoneBr,Hood_Timber,Hood_Veenker
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,0,0,0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,0,0,0,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,0,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,0,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,1,0,0,0,0,0,0,0,0,0


In [16]:
# Create dummies
condi1dum = pd.get_dummies(house['Condition1'])

# Rename columns
for i in condi1dum.columns:
    condi1dum.rename(columns={i: 'Condi1_'+str(i)}, inplace=True)
    
# Concatenate dummy variables into main df: house
house = pd.concat([house, condi1dum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Hood_Veenker,Condi1_Artery,Condi1_Feedr,Condi1_Norm,Condi1_PosA,Condi1_PosN,Condi1_RRAe,Condi1_RRAn,Condi1_RRNe,Condi1_RRNn
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,1,0,0,0,0,0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,1,0,1,0,0,0,0,0,0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,1,0,0,0,0,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,1,0,0,0,0,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,1,0,0,0,0,0,0


In [17]:
# Create dummies
bldgtypedum = pd.get_dummies(house['BldgType'])

# Rename columns
bldgtypedum.columns = ['BldgType_1fam','BldgType_2famconv','BldgType_duplex','BldgType_twnhsIns','BldgType_twnhsEnd',]

# Concatenate dummy variables into main df: house
house = pd.concat([house, bldgtypedum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Condi1_PosN,Condi1_RRAe,Condi1_RRAn,Condi1_RRNe,Condi1_RRNn,BldgType_1fam,BldgType_2famconv,BldgType_duplex,BldgType_twnhsIns,BldgType_twnhsEnd
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,1,0,0,0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,1,0,0,0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,1,0,0,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,1,0,0,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,1,0,0,0,0


In [18]:
# Create dummies
houseStyleDum = pd.get_dummies(house['HouseStyle'])

# Rename columns
for i in houseStyleDum.columns:
    houseStyleDum.rename(columns={i: 'HouStyle_'+str(i)}, inplace=True)

# Concatenate dummy variables into main df: house
house = pd.concat([house, houseStyleDum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,BldgType_twnhsIns,BldgType_twnhsEnd,HouStyle_1.5Fin,HouStyle_1.5Unf,HouStyle_1Story,HouStyle_2.5Fin,HouStyle_2.5Unf,HouStyle_2Story,HouStyle_SFoyer,HouStyle_SLvl
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,0,1,0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,1,0,0,0,0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,1,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,1,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,1,0,0


In [19]:
# Create dummies
roofStyleDum = pd.get_dummies(house['RoofStyle'])

# Rename columns
for i in roofStyleDum.columns:
    roofStyleDum.rename(columns={i: 'RoofStyle_'+str(i)}, inplace=True)

# Concatenate dummy variables into main df: house
house = pd.concat([house, roofStyleDum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,HouStyle_2.5Unf,HouStyle_2Story,HouStyle_SFoyer,HouStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,1,0,0,0,1,0,0,0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,1,0,0,0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,1,0,0,0,1,0,0,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,1,0,0,0,1,0,0,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,1,0,0,0,1,0,0,0,0


In [20]:
# Create dummies
roofMatlDum = pd.get_dummies(house['RoofMatl'])

# Rename columns
for i in roofMatlDum.columns:
    roofMatlDum.rename(columns={i: 'RoofMatl_'+str(i)}, inplace=True)

# Concatenate dummy variables into main df: house
house = pd.concat([house, roofMatlDum], axis=1, join='outer')
house.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,1,0,0,0,0,0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,1,0,0,0,0,0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,1,0,0,0,0,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,1,0,0,0,0,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,1,0,0,0,0,0,0


In [21]:
# Create dummies
ExteriorDum = pd.get_dummies(house['Exterior1st'])

# Rename columns
for i in ExteriorDum.columns:
    ExteriorDum.rename(columns={i: 'Exter1_'+str(i)}, inplace=True)
    
# Concatenate dummy variables into main df: house
house = pd.concat([house, ExteriorDum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Exter1_CemntBd,Exter1_HdBoard,Exter1_ImStucc,Exter1_MetalSd,Exter1_Plywood,Exter1_Stone,Exter1_Stucco,Exter1_VinylSd,Exter1_Wd Sdng,Exter1_WdShing
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,0,1,0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,1,0,0,0,0,0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,1,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,0,1,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,1,0,0


In [22]:
# Create dummies
MasVnrDum = pd.get_dummies(house['MasVnrType'])

# Rename columns
for i in MasVnrDum.columns:
    MasVnrDum.rename(columns={i: 'MasVnr_'+str(i)}, inplace=True)

# Concatenate dummy variables into main df: house
house = pd.concat([house, MasVnrDum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Exter1_Plywood,Exter1_Stone,Exter1_Stucco,Exter1_VinylSd,Exter1_Wd Sdng,Exter1_WdShing,MasVnr_BrkCmn,MasVnr_BrkFace,MasVnr_None,MasVnr_Stone
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,1,0,0,0,1,0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,0,0,1,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,1,0,0,0,1,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,1,0,0,0,1,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,1,0,0,0,1,0,0


In [23]:
# Create dummies
foundDum = pd.get_dummies(house['Foundation'])

# Rename columns
for i in foundDum.columns:
    foundDum.rename(columns={i: 'Fndtn_'+str(i)}, inplace=True)
    
# Concatenate dummy variables into main df: house
house = pd.concat([house, foundDum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MasVnr_BrkCmn,MasVnr_BrkFace,MasVnr_None,MasVnr_Stone,Fndtn_BrkTil,Fndtn_CBlock,Fndtn_PConc,Fndtn_Slab,Fndtn_Stone,Fndtn_Wood
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,1,0,0,0,0,1,0,0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,1,0,0,1,0,0,0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,1,0,0,0,0,1,0,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,1,0,1,0,0,0,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,1,0,0,0,0,1,0,0,0


In [24]:
# Create dummies
utilDum = pd.get_dummies(house['Utilities'])

# Rename columns
utilDum.columns = ['Util_All', 'Util_NoSewa']

# Concatenate dummy variables into main df: house
house = pd.concat([house, utilDum], axis=1, join='outer')

,Util_All,Util_NoSewa
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [25]:
# Create dummies
heatingDum = pd.get_dummies(house['Heating'])

# Rename columns
for i in heatingDum.columns:
    heatingDum.rename(columns={i: 'Heatg_'+str(i)}, inplace=True)
    
# Concatenate dummy variables into main df: house
house = pd.concat([house, heatingDum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Fndtn_Stone,Fndtn_Wood,Util_All,Util_NoSewa,Heatg_Floor,Heatg_GasA,Heatg_GasW,Heatg_Grav,Heatg_OthW,Heatg_Wall
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,1,0,0,1,0,0,0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,1,0,0,1,0,0,0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,1,0,0,1,0,0,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,1,0,0,1,0,0,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,1,0,0,1,0,0,0,0


In [26]:
# Create dummies
electricDum = pd.get_dummies(house['Electrical'])

# Rename columns
for i in electricDum.columns:
    electricDum.rename(columns={i: 'Elec_'+str(i)}, inplace=True)

# Concatenate dummy variables into main df: house
house = pd.concat([house, electricDum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Heatg_GasA,Heatg_GasW,Heatg_Grav,Heatg_OthW,Heatg_Wall,Elec_FuseA,Elec_FuseF,Elec_FuseP,Elec_Mix,Elec_SBrkr
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,1,0,0,0,0,0,0,0,0,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,1,0,0,0,0,0,0,0,0,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,1,0,0,0,0,0,0,0,0,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,1,0,0,0,0,0,0,0,0,1
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,1,0,0,0,0,0,0,0,0,1


In [27]:
# Create dummies
functionDum = pd.get_dummies(house['Functional'])

# Rename columns
for i in functionDum.columns:
    functionDum.rename(columns={i: 'Func_'+str(i)}, inplace=True)

# Concatenate dummy variables into main df: house
house = pd.concat([house, functionDum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Elec_FuseP,Elec_Mix,Elec_SBrkr,Func_Maj1,Func_Maj2,Func_Min1,Func_Min2,Func_Mod,Func_Sev,Func_Typ
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,1,0,0,0,0,0,0,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,1,0,0,0,0,0,0,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,1,0,0,0,0,0,0,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,1,0,0,0,0,0,0,1
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,1,0,0,0,0,0,0,1


In [28]:
# Create dummies
garagetypeDum = pd.get_dummies(house['GarageType'])

# Rename columns
for i in garagetypeDum.columns:
    garagetypeDum.rename(columns={i: 'GarType_'+str(i)}, inplace=True)

# Concatenate dummy variables into main df: house
house = pd.concat([house, garagetypeDum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Func_Min2,Func_Mod,Func_Sev,Func_Typ,GarType_2Types,GarType_Attchd,GarType_Basment,GarType_BuiltIn,GarType_CarPort,GarType_Detchd
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,1,0,1,0,0,0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,1,0,1,0,0,0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,1,0,1,0,0,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,1,0,0,0,0,0,1
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,1,0,1,0,0,0,0


In [29]:
# Create dummies
lotconDum = pd.get_dummies(house['LotConfig'])

# Rename columns
lotconDum.columns = ['LotCon_Corner','LotCon_CulDSac','LotCon_Front2','LotCon_Front3','LotCon_Inside',]

# Concatenate dummy variables into main df: house
house = pd.concat([house, lotconDum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,GarType_Attchd,GarType_Basment,GarType_BuiltIn,GarType_CarPort,GarType_Detchd,LotCon_Corner,LotCon_CulDSac,LotCon_Front2,LotCon_Front3,LotCon_Inside
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,1,0,0,0,0,0,0,0,0,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,1,0,0,0,0,0,0,1,0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,1,0,0,0,0,0,0,0,0,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,1,1,0,0,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,1,0,0,0,0,0,0,1,0,0


In [30]:
# Create dummies
pavedDriveDum = pd.get_dummies(house['PavedDrive'])

# Rename columns
pavedDriveDum.columns = ['Paved_DirtGrav', 'Paved_part', 'Paved_paved']

# Concatenate dummy variables into main df: house
house = pd.concat([house, pavedDriveDum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,GarType_CarPort,GarType_Detchd,LotCon_Corner,LotCon_CulDSac,LotCon_Front2,LotCon_Front3,LotCon_Inside,Paved_DirtGrav,Paved_part,Paved_paved
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,1,0,0,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,1,0,0,0,0,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,1,0,0,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,1,1,0,0,0,0,0,0,1
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,1,0,0,0,0,1


In [31]:
# Create dummies
miscfeatDum = pd.get_dummies(house['MiscFeature'])

# Rename 1 column
miscfeatDum.rename(columns={'TenC': 'TenCrt'}, inplace=True)

# Rename all by looping and adding suffix
for i in miscfeatDum.columns:
    miscfeatDum.rename(columns={i: 'MiscFeat_'+str(i)}, inplace=True)

# Concatenate dummy variables into main df: house
house = pd.concat([house, miscfeatDum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,LotCon_Front2,LotCon_Front3,LotCon_Inside,Paved_DirtGrav,Paved_part,Paved_paved,MiscFeat_Gar2,MiscFeat_Othr,MiscFeat_Shed,MiscFeat_TenCrt
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,1,0,0,1,0,0,0,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,1,0,0,0,0,1,0,0,0,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,1,0,0,1,0,0,0,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,1,0,0,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,1,0,0,0,0,1,0,0,0,0


In [32]:
# Get dummies
saletypeDum = pd.get_dummies(house['SaleType'])

# Rename columns (more descriptive)
saletypeDum.columns = ['CrtOficDeedEst', 'CashWarrDeed', 'Con15dpReg', 'ConLwDp','ConLwInt',
                      'ConLwdpLwInt','New','Other', 'WarrDeed']
# Add prefix to columns
for i in saletypeDum.columns:
    saletypeDum.rename(columns={i: 'SaleType_'+str(i)}, inplace=True)

# Concatenate dummy variables into main df: house
house = pd.concat([house, saletypeDum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeat_TenCrt,SaleType_CrtOficDeedEst,SaleType_CashWarrDeed,SaleType_Con15dpReg,SaleType_ConLwDp,SaleType_ConLwInt,SaleType_ConLwdpLwInt,SaleType_New,SaleType_Other,SaleType_WarrDeed
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,0,0,0,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,0,0,0,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,0,0,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,0,0,1
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,0,0,1


In [33]:
# Get dummies
saleconDum = pd.get_dummies(house['SaleCondition'])

# Rename columns
for i in saleconDum.columns:
    saleconDum.rename(columns={i: 'SaleCond_'+str(i)}, inplace=True)

# Concatenate dummy variables into main df: house
house = pd.concat([house, saleconDum], axis=1, join='outer')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,SaleType_ConLwdpLwInt,SaleType_New,SaleType_Other,SaleType_WarrDeed,SaleCond_Abnorml,SaleCond_AdjLand,SaleCond_Alloca,SaleCond_Family,SaleCond_Normal,SaleCond_Partial
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,1,0,0,0,0,1,0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,1,0,0,0,0,1,0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,1,0,0,0,0,1,0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,1,1,0,0,0,0,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,1,0,0,0,0,1,0


# Converting level based variables to numeric

In [34]:
# Map individual columns below to produce numeric values from unicode-based levels
house['ExterQual'] = house['ExterQual'].map({'Gd': 4, 'TA': 3, 'Ex': 5, 'Fa': 2})
house['ExterCond'] = house['ExterCond'].map({'Gd': 4, 'TA': 3, 'Ex': 5, 'Fa': 2, 'Po': 1})
house['BsmtQual'] = house['BsmtQual'].map({'Gd': 4, 'TA': 3, 'Ex': 5, 'Fa': 2, 'Po': 1, np.nan: 0})
house['BsmtCond'] = house['BsmtCond'].map({'Gd': 4, 'TA': 3, 'Ex': 5, 'Fa': 2, 'Po': 1, np.nan: 0})
house['BsmtFinType1'] = house['BsmtFinType1'].map({'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ':2, 'Unf': 1, np.nan: 0})
house['HeatingQC'] = house['HeatingQC'].map({'Gd': 4, 'TA': 3, 'Ex': 5, 'Fa': 2, 'Po': 1, np.nan: 0})
house['CentralAir'] = house['CentralAir'].map({'Y': 1, 'N': 0})
house['KitchenQual'] = house['KitchenQual'].map({'Gd': 4, 'TA': 3, 'Ex': 5, 'Fa': 2, 'Po': 1, np.nan: 0})
house['FireplaceQu'] = house['FireplaceQu'].map({'Gd': 4, 'TA': 3, 'Ex': 5, 'Fa': 2, 'Po': 1, np.nan: 0})
house['GarageFinish'] = house['GarageFinish'].map({'Fin': 3, 'RFn': 2, 'Unf': 1, np.nan: 0})
house['GarageQual'] = house['GarageQual'].map({'Gd': 4, 'TA': 3, 'Ex': 5, 'Fa': 2, 'Po': 1, np.nan: 0})
house['GarageCond'] = house['GarageCond'].map({'Gd': 4, 'TA': 3, 'Ex': 5, 'Fa': 2, 'Po': 1, np.nan: 0})
house['PoolQC'] = house['PoolQC'].map({'Gd': 4, 'TA': 3, 'Ex': 5, 'Fa': 2, 'Po': 1, np.nan: 0})
house['Fence'] = house['Fence'].map({'MnPrv': 3, 'GdWo': 2, 'GdPrv': 4, 'MnWw': 1, np.nan: 0})
house['LandSlope'] = house['LandSlope'].map({'Gtl': 1, 'Mod': 2, 'Sev': 3})
house['BsmtExposure'] = house['BsmtExposure'].map({'Gd': 3,'Av': 2, 'Mn': 1,'No': 0, np.nan: 0})

# Replace null values with 0
house['LotFrontage'] = house['LotFrontage'].fillna(value=0)
house['Alley'] = house['Alley'].fillna(value=0)
house['MasVnrArea'].fillna(value=0, inplace=True)

#Replace null garage yr built to that of structure built
house['GarageYrBlt'].fillna(value=house['YearBuilt'], inplace=True)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,SaleType_ConLwdpLwInt,SaleType_New,SaleType_Other,SaleType_WarrDeed,SaleCond_Abnorml,SaleCond_AdjLand,SaleCond_Alloca,SaleCond_Family,SaleCond_Normal,SaleCond_Partial
0,1,60,RL,65.0,8450,Pave,0,Reg,Lvl,AllPub,...,0,0,0,1,0,0,0,0,1,0
1,2,20,RL,80.0,9600,Pave,0,Reg,Lvl,AllPub,...,0,0,0,1,0,0,0,0,1,0
2,3,60,RL,68.0,11250,Pave,0,IR1,Lvl,AllPub,...,0,0,0,1,0,0,0,0,1,0
3,4,70,RL,60.0,9550,Pave,0,IR1,Lvl,AllPub,...,0,0,0,1,1,0,0,0,0,0
4,5,60,RL,84.0,14260,Pave,0,IR1,Lvl,AllPub,...,0,0,0,1,0,0,0,0,1,0


# The house Dataframe will be renamed and further split into the required datasets

In [89]:
# Make a copy of house and assign to df
df = house.copy()

# Split test and train data by years: pre-2010, 2010
test = df[df['YrSold']==2010]
train = df[df['YrSold']<2010]

# Assign to traindf and testdf a filtered version of test and train data
# of only residential properties
traindf = train[(train['MSZon_Resi_FloatVillage']==1) | (train['MSZon_Resi_HighDensity']==1) | (train['MSZon_Resi_LowDensity']==1) | (train['MSZon_Resi_MedDensity']==1)]
testdf = test[(test['MSZon_Resi_FloatVillage']==1) | (test['MSZon_Resi_HighDensity']==1) | (test['MSZon_Resi_LowDensity']==1) | (test['MSZon_Resi_MedDensity']==1)]

## Residential Properties
The dataset contains a number of categories of properties within the MSZoning variable. Residential properties start with an R, with options of RH, RL, RP, RM. We will separate those properties considered residential and place them in a separate dataframe

# Determination of Predictors
## Fixed characteristics
The characteristics of a house determined to be fixed will be run through a correlation matrix to determine predictors.


### Converted to Dummy Variables
* MSSubclass
* MSZoning

* LotShape
* LandContour
* Utilities
* LotConfig
* LandSlope
* Neighborhood
* Condition1
* BldgType
* HouseStyle
* MasVnrType
* Foundation
* GarageType



### Ordinary Variables
* LotArea
* LotFrontage
* YearBuilt
* MasVnrArea
* BsmtExposure
* TotalBsmtSF
* 1stFlrSF
* 2ndFlrSF
* GrLivArea
* BsmtFullBath
* BsmtHalfBath
* Bedroom
* Kitchen
* Fireplaces
* GarageYrBlt
* GarageCars
* GarageArea
* WoodDeckSF
* OpenPorchSF
* EnclosedPorch
* 3SsnPorch
* ScreenPorch
* PoolArea




## Renovate-able characteristics
* OverallQual
* OverallCond
* ExterQual
* ExterCond
* BsmtQual
* BsmtCond
* BsmtFinType1
* Heating
* HeatingQC
* CentralAir
* Electrical : Dummy Variables
* KitchenQual
* Functional :Dummy
* FireplaceQu
* GarageFinish
* GarageQual
* GarageCond
* PavedDrive : Dummy
* PoolQC
* Fence
* TotRmsAbvGrd
* FullBath
* HalfBath
* RoofStyle
* RoofMatl
* Exterior1st

In [133]:
# Initialize training variables
fixedX_train = train.loc[:,['MSSub_20', 'MSSub_30', 'MSSub_40', 'MSSub_45', 'MSSub_50', 'MSSub_60', 'MSSub_70', 'MSSub_75',
                  'MSSub_80', 'MSSub_85', 'MSSub_90', 'MSSub_120', 'MSSub_160', 'MSSub_180', 'MSSub_190',
                  'MSZon_Commercial', 'MSZon_Resi_FloatVillage', 'MSZon_Resi_HighDensity', 'MSZon_Resi_LowDensity',
                  'MSZon_Resi_MedDensity','LotSha_SlightIrregular', 'LotSha_ModIrregular', 'LotSha_Irregular',
                  'LotSha_Regular','LContour_Banked', 'LContour_Hillside', 'LContour_Depression','LContour_Flat',
                  'Util_All', 'Util_NoSewa','LotCon_Corner', 'LotCon_CulDSac', 'LotCon_Front2','LotCon_Front3',
                  'LotCon_Inside', 'LandSlope', 'Hood_Blmngtn', 'Hood_Blueste', 'Hood_BrDale', 'Hood_BrkSide',
                  'Hood_ClearCr', 'Hood_CollgCr', 'Hood_Crawfor', 'Hood_Edwards','Hood_Gilbert', 'Hood_IDOTRR', 
                  'Hood_MeadowV', 'Hood_Mitchel','Hood_NAmes', 'Hood_NPkVill', 'Hood_NWAmes', 'Hood_NoRidge',
                  'Hood_NridgHt', 'Hood_OldTown', 'Hood_SWISU', 'Hood_Sawyer','Hood_SawyerW', 'Hood_Somerst', 'Hood_StoneBr', 
                  'Hood_Timber','Hood_Veenker','Condi1_Artery', 'Condi1_Feedr','Condi1_Norm', 'Condi1_PosA', 
                  'Condi1_PosN', 'Condi1_RRAe','Condi1_RRAn', 'Condi1_RRNe', 'Condi1_RRNn','BldgType_1fam', 
                  'BldgType_2famconv', 'BldgType_duplex','BldgType_twnhsIns', 'BldgType_twnhsEnd','HouStyle_1.5Fin',
                  'HouStyle_1.5Unf','HouStyle_1Story', 'HouStyle_2.5Fin', 'HouStyle_2.5Unf','HouStyle_2Story', 
                  'HouStyle_SFoyer', 'HouStyle_SLvl', 'MasVnr_BrkCmn', 'MasVnr_BrkFace','MasVnr_None', 'MasVnr_Stone',
                  'Fndtn_BrkTil', 'Fndtn_CBlock', 'Fndtn_PConc', 'Fndtn_Slab','Fndtn_Stone', 'Fndtn_Wood',
                  'GarType_2Types','GarType_Attchd', 'GarType_Basment', 'GarType_BuiltIn','GarType_CarPort', 
                  'GarType_Detchd','Street_Gravel','Street_Paved','LotFrontage', 'LotArea','YearBuilt','MasVnrArea',
                  'BsmtExposure','TotalBsmtSF','BsmtFullBath', 'BsmtHalfBath','1stFlrSF', '2ndFlrSF','GrLivArea',
                  'BedroomAbvGr', 'KitchenAbvGr','Fireplaces','GarageYrBlt','GarageCars','GarageArea','WoodDeckSF',
                  'OpenPorchSF', 'EnclosedPorch','3SsnPorch', 'ScreenPorch', 'PoolArea','Alley_Gravel', 'Alley_Paved']]

# Initialize y_train
y_train=train['SalePrice']

In [134]:
# Initialize test variables
fixedX_test = test.loc[:,['MSSub_20', 'MSSub_30', 'MSSub_40', 'MSSub_45', 'MSSub_50', 'MSSub_60', 'MSSub_70', 'MSSub_75',
                  'MSSub_80', 'MSSub_85', 'MSSub_90', 'MSSub_120', 'MSSub_160', 'MSSub_180', 'MSSub_190',
                  'MSZon_Commercial', 'MSZon_Resi_FloatVillage', 'MSZon_Resi_HighDensity', 'MSZon_Resi_LowDensity',
                  'MSZon_Resi_MedDensity','LotSha_SlightIrregular', 'LotSha_ModIrregular', 'LotSha_Irregular',
                  'LotSha_Regular','LContour_Banked', 'LContour_Hillside', 'LContour_Depression','LContour_Flat',
                  'Util_All', 'Util_NoSewa','LotCon_Corner', 'LotCon_CulDSac', 'LotCon_Front2','LotCon_Front3',
                  'LotCon_Inside', 'LandSlope', 'Hood_Blmngtn', 'Hood_Blueste', 'Hood_BrDale', 'Hood_BrkSide',
                  'Hood_ClearCr', 'Hood_CollgCr', 'Hood_Crawfor', 'Hood_Edwards','Hood_Gilbert', 'Hood_IDOTRR', 
                  'Hood_MeadowV', 'Hood_Mitchel','Hood_NAmes', 'Hood_NPkVill', 'Hood_NWAmes', 'Hood_NoRidge',
                  'Hood_NridgHt', 'Hood_OldTown', 'Hood_SWISU', 'Hood_Sawyer','Hood_SawyerW', 'Hood_Somerst', 'Hood_StoneBr', 
                  'Hood_Timber','Hood_Veenker','Condi1_Artery', 'Condi1_Feedr','Condi1_Norm', 'Condi1_PosA', 
                  'Condi1_PosN', 'Condi1_RRAe','Condi1_RRAn', 'Condi1_RRNe', 'Condi1_RRNn','BldgType_1fam', 
                  'BldgType_2famconv', 'BldgType_duplex','BldgType_twnhsIns', 'BldgType_twnhsEnd','HouStyle_1.5Fin',
                  'HouStyle_1.5Unf','HouStyle_1Story', 'HouStyle_2.5Fin', 'HouStyle_2.5Unf','HouStyle_2Story', 
                  'HouStyle_SFoyer', 'HouStyle_SLvl', 'MasVnr_BrkCmn', 'MasVnr_BrkFace','MasVnr_None', 'MasVnr_Stone',
                  'Fndtn_BrkTil', 'Fndtn_CBlock', 'Fndtn_PConc', 'Fndtn_Slab','Fndtn_Stone', 'Fndtn_Wood',
                  'GarType_2Types','GarType_Attchd', 'GarType_Basment', 'GarType_BuiltIn','GarType_CarPort', 
                  'GarType_Detchd','Street_Gravel','Street_Paved','LotFrontage', 'LotArea','YearBuilt','MasVnrArea',
                  'BsmtExposure','TotalBsmtSF','BsmtFullBath', 'BsmtHalfBath','1stFlrSF', '2ndFlrSF','GrLivArea',
                  'BedroomAbvGr', 'KitchenAbvGr','Fireplaces','GarageYrBlt','GarageCars','GarageArea','WoodDeckSF',
                  'OpenPorchSF', 'EnclosedPorch','3SsnPorch', 'ScreenPorch', 'PoolArea','Alley_Gravel', 'Alley_Paved']]

# Initialize y_test
y_test = test['SalePrice']

In [144]:
# Import packages
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFE
from scipy.stats import zscore

# Standardize train and test X
standFixed = fixedX_train.apply(zscore, axis=1)
standFixed_test = fixedX_test.apply(zscore, axis=1)

# Assign model to ridge regression and fit to training data
model = Ridge(alpha=3)
model.fit(standFixed, y_train)

# Fit RFE to current model
rfe = RFE(model)
rfe = rfe.fit(standFixed, y_train)

In [136]:
# Print results of RFE
print(rfe.support_)
print(rfe.ranking_)
standFixed.columns[rfe.support_].values

array(['MSSub_20', 'MSSub_50', 'MSSub_70', 'MSSub_75', 'MSSub_80',
       'MSSub_90', 'MSSub_160', 'MSSub_190', 'MSZon_Commercial',
       'MSZon_Resi_LowDensity', 'LotSha_Regular', 'LContour_Flat',
       'Util_All', 'LotCon_Corner', 'LotCon_Front2', 'Hood_Blmngtn',
       'Hood_BrDale', 'Hood_ClearCr', 'Hood_Edwards', 'Hood_IDOTRR',
       'Hood_Mitchel', 'Hood_NAmes', 'Hood_NWAmes', 'Hood_OldTown',
       'Hood_SWISU', 'Hood_Sawyer', 'Condi1_Artery', 'Condi1_Feedr',
       'BldgType_2famconv', 'BldgType_duplex', 'HouStyle_1.5Fin',
       'HouStyle_1Story', 'HouStyle_2.5Unf', 'MasVnr_BrkCmn',
       'MasVnr_BrkFace', 'Fndtn_BrkTil', 'Fndtn_CBlock', 'Fndtn_Stone',
       'GarType_2Types', 'GarType_CarPort', 'GarType_Detchd',
       'LotFrontage', 'LotArea', 'YearBuilt', 'MasVnrArea', 'BsmtExposure',
       'TotalBsmtSF', 'BsmtHalfBath', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
       'BedroomAbvGr', 'KitchenAbvGr', 'GarageYrBlt', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'Enclosed

In [145]:
# Import score packages
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# Calculate cross val score and mean
scores = cross_val_score(model,standFixed_test,y=y_test)
print('All scores: {}\n Mean:{}'.format(scores, scores.mean()))

All scores: [ 0.62879248  0.41551033  0.65223628]
 Mean:0.565513029677


In [146]:
# Generate predicted value based on model
y_pred = model.predict(standFixed_test)

# Root mean squared error
np.sqrt(mean_squared_error(y_test, y_pred))

50665.314686735706

# Attempt to create an improved model
* By removing predictors not included in above returned list from RFE, hope for a better performing model
* Score appears to decrease, meaning above model is better. Model above is used.

In [43]:
# Assign reduced number of predictors returned by RFE: Fixedfin_train, Fixedfin_test
Fixedfin_train = train.loc[:,['MSSub_20', 'MSSub_50', 'MSSub_70', 'MSSub_75', 'MSSub_80',
       'MSSub_90', 'MSSub_160', 'MSSub_190', 'MSZon_Commercial',
       'MSZon_Resi_LowDensity', 'LotSha_Regular', 'LContour_Flat',
       'Util_All', 'LotCon_Corner', 'LotCon_Front2', 'Hood_Blmngtn',
       'Hood_BrDale', 'Hood_ClearCr', 'Hood_Edwards', 'Hood_IDOTRR',
       'Hood_Mitchel', 'Hood_NAmes', 'Hood_NWAmes', 'Hood_OldTown',
       'Hood_SWISU', 'Hood_Sawyer', 'Condi1_Artery', 'Condi1_Feedr',
       'BldgType_2famconv', 'BldgType_duplex', 'HouStyle_1.5Fin',
       'HouStyle_1Story', 'HouStyle_2.5Unf', 'MasVnr_BrkCmn',
       'MasVnr_BrkFace', 'Fndtn_BrkTil', 'Fndtn_CBlock', 'Fndtn_Stone',
       'GarType_2Types', 'GarType_CarPort', 'GarType_Detchd',
       'LotFrontage', 'LotArea', 'YearBuilt', 'MasVnrArea', 'BsmtExposure',
       'TotalBsmtSF', 'BsmtHalfBath', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
       'BedroomAbvGr', 'KitchenAbvGr', 'GarageYrBlt', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'Alley_Gravel', 'Alley_Paved']]

Fixedfin_test= test.loc[:,['MSSub_20', 'MSSub_50', 'MSSub_70', 'MSSub_75', 'MSSub_80',
       'MSSub_90', 'MSSub_160', 'MSSub_190', 'MSZon_Commercial',
       'MSZon_Resi_LowDensity', 'LotSha_Regular', 'LContour_Flat',
       'Util_All', 'LotCon_Corner', 'LotCon_Front2', 'Hood_Blmngtn',
       'Hood_BrDale', 'Hood_ClearCr', 'Hood_Edwards', 'Hood_IDOTRR',
       'Hood_Mitchel', 'Hood_NAmes', 'Hood_NWAmes', 'Hood_OldTown',
       'Hood_SWISU', 'Hood_Sawyer', 'Condi1_Artery', 'Condi1_Feedr',
       'BldgType_2famconv', 'BldgType_duplex', 'HouStyle_1.5Fin',
       'HouStyle_1Story', 'HouStyle_2.5Unf', 'MasVnr_BrkCmn',
       'MasVnr_BrkFace', 'Fndtn_BrkTil', 'Fndtn_CBlock', 'Fndtn_Stone',
       'GarType_2Types', 'GarType_CarPort', 'GarType_Detchd',
       'LotFrontage', 'LotArea', 'YearBuilt', 'MasVnrArea', 'BsmtExposure',
       'TotalBsmtSF', 'BsmtHalfBath', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
       'BedroomAbvGr', 'KitchenAbvGr', 'GarageYrBlt', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'Alley_Gravel', 'Alley_Paved']]

In [44]:
# Standardize new set of predictors
standFixed = Fixedfin_train.apply(zscore, axis=1)
standFixed_test = Fixedfin_test.apply(zscore, axis=1)

# Instantiate model
model = Ridge(alpha=3)

# Fit model to training data
model.fit(standFixed, y_train)

Ridge(alpha=3, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [147]:
# Generate cross val score
rfescores = cross_val_score(model,standFixed_test,y=y_test)
print('All scores: {}\n Mean:{}'.format(rfescores, rfescores.mean()))

All scores: [ 0.62879248  0.41551033  0.65223628]
 Mean:0.565513029677


In [148]:
# Generate predicted values
y_pred = model.predict(standFixed_test)
# Generate RMSE for scoring
np.sqrt(mean_squared_error(y_test, y_pred))

50665.314686735706

# Use of GridSearch to improve parameters of model
* Results of GridSearch.best_params_ used above

In [47]:
from sklearn.model_selection import GridSearchCV

params = {'alpha': [0.1,0.2,0.4,0.6,0.8,1,3,5,8,10],
          'solver': ['auto', 'lsqr', 'sag']
    
}

search = GridSearchCV(model, params, n_jobs=3)

In [48]:
search.fit(fixedX_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Ridge(alpha=3, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'alpha': [0.1, 0.2, 0.4, 0.6, 0.8, 1, 3, 5, 8, 10], 'solver': ['auto', 'lsqr', 'sag']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [49]:
# Print results of Gridsearch for use
print(search.best_params_)
print(search.best_score_)

{'alpha': 3, 'solver': 'auto'}
0.780240218403


# Residential Properties
The dataset contains a number of categories of properties within the MSZoning variable. Residential properties start with an R, with options of RH, RL, RP, RM. We will separate those properties considered residential and place them in a separate dataframe

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Determine any value of *changeable* property characteristics unexplained by the *fixed* ones.

---

Now that you have a model that estimates the price of a house based on its static characteristics, we can move forward with part 2 and 3 of the plan: what are the costs/benefits of quality, condition, and renovations?

There are two specific requirements for these estimates:
1. The estimates of effects must be in terms of dollars added or subtracted from the house value. 
2. The effects must be on the variance in price remaining from the first model.

The residuals from the first model (training and testing) represent the variance in price unexplained by the fixed characteristics. Of that variance in price remaining, how much of it can be explained by the easy-to-change aspects of the property?

---

**Your goals:**
1. Evaluate the effect in dollars of the renovate-able features. 
- How would your company use this second model and its coefficients to determine whether they should buy a property or not? Explain how the company can use the two models you have built to determine if they can make money. 
- Investigate how much of the variance in price remaining is explained by these features.
- Do you trust your model? Should it be used to evaluate which properties to buy and fix up?

# The below workings are incomplete

In [124]:
# Commented out: result of returned RFE
# renoX_train = train.loc[:,['FireplaceQu', 'PoolQC', 'BsmtFinType1',  'Fence',
#                            'RoofStyle_Gambrel', 'Exter1_CemntBd']]

# renoX_test = test.loc[:,['FireplaceQu', 'PoolQC', 'BsmtFinType1',  'Fence',
#                            'RoofStyle_Gambrel',  'Exter1_CemntBd']]

# Assign renovatable characteristics to renoX_train and renoX_test
renoX_train = train.loc[:,['OverallQual', 'OverallCond','FireplaceQu','GarageFinish','GarageQual', 'GarageCond', 'PoolQC',
                         'ExterQual', 'ExterCond','BsmtQual', 'BsmtCond','BsmtFinType1','Heatg_Floor', 'Heatg_GasA', 
                         'Heatg_GasW', 'Heatg_Grav','Heatg_OthW', 'Heatg_Wall','HeatingQC','CentralAir', 'Elec_FuseA',
                         'Elec_FuseF', 'Elec_FuseP', 'Elec_Mix','Elec_SBrkr','KitchenQual', 'Func_Maj1', 'Func_Maj2', 
                         'Func_Min1', 'Func_Min2','Func_Mod', 'Func_Sev', 'Func_Typ','Paved_DirtGrav','Paved_part', 
                         'Paved_paved', 'Fence', 'TotRmsAbvGrd','FullBath', 'HalfBath','RoofStyle_Flat', 
                         'RoofStyle_Gable','RoofStyle_Gambrel', 'RoofStyle_Hip', 'RoofStyle_Mansard','RoofStyle_Shed',
                         'RoofMatl_ClyTile', 'RoofMatl_CompShg','RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll',
                         'RoofMatl_Tar&Grv', 'RoofMatl_WdShake', 'RoofMatl_WdShngl','Exter1_AsbShng', 'Exter1_AsphShn',
                         'Exter1_BrkComm','Exter1_BrkFace', 'Exter1_CBlock','Exter1_CemntBd','Exter1_HdBoard', 
                         'Exter1_ImStucc', 'Exter1_MetalSd','Exter1_Plywood', 'Exter1_Stone', 'Exter1_Stucco', 
                         'Exter1_VinylSd','Exter1_Wd Sdng', 'Exter1_WdShing']]

renoX_test = test.loc[:,['OverallQual', 'OverallCond','FireplaceQu','GarageFinish','GarageQual', 'GarageCond', 'PoolQC',
                         'ExterQual', 'ExterCond','BsmtQual', 'BsmtCond','BsmtFinType1','Heatg_Floor', 'Heatg_GasA', 
                         'Heatg_GasW', 'Heatg_Grav','Heatg_OthW', 'Heatg_Wall','HeatingQC','CentralAir', 'Elec_FuseA',
                         'Elec_FuseF', 'Elec_FuseP', 'Elec_Mix','Elec_SBrkr','KitchenQual', 'Func_Maj1', 'Func_Maj2', 
                         'Func_Min1', 'Func_Min2','Func_Mod', 'Func_Sev', 'Func_Typ','Paved_DirtGrav','Paved_part', 
                         'Paved_paved', 'Fence', 'TotRmsAbvGrd','FullBath', 'HalfBath','RoofStyle_Flat', 
                         'RoofStyle_Gable','RoofStyle_Gambrel', 'RoofStyle_Hip', 'RoofStyle_Mansard','RoofStyle_Shed',
                         'RoofMatl_ClyTile', 'RoofMatl_CompShg','RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll',
                         'RoofMatl_Tar&Grv', 'RoofMatl_WdShake', 'RoofMatl_WdShngl','Exter1_AsbShng', 'Exter1_AsphShn',
                         'Exter1_BrkComm','Exter1_BrkFace', 'Exter1_CBlock','Exter1_CemntBd','Exter1_HdBoard', 
                         'Exter1_ImStucc', 'Exter1_MetalSd','Exter1_Plywood', 'Exter1_Stone', 'Exter1_Stucco', 
                         'Exter1_VinylSd','Exter1_Wd Sdng', 'Exter1_WdShing']]


#df.filter(like='Heatg').columns.values
#standrenoX_train.select_dtypes(include=[np.float64]).describe()
#renoX_train.isnull().sum().sort_values(ascending=False)

,OverallQual,OverallCond,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC,ExterQual,ExterCond,BsmtQual,...,Exter1_CemntBd,Exter1_HdBoard,Exter1_ImStucc,Exter1_MetalSd,Exter1_Plywood,Exter1_Stone,Exter1_Stucco,Exter1_VinylSd,Exter1_Wd Sdng,Exter1_WdShing
16,6,7,3,3,3,3,0,3,3,3,...,0,0,0,0,0,0,0,0,1,0
24,5,8,3,1,3,3,0,3,4,3,...,0,0,0,0,1,0,0,0,0,0
26,5,7,0,1,3,3,0,3,3,3,...,0,0,0,0,0,0,0,0,1,0
27,8,5,4,2,3,3,0,4,3,5,...,0,0,0,0,0,0,0,1,0,0
33,5,5,4,2,3,3,0,3,3,3,...,0,0,0,0,0,0,0,0,0,0


In [128]:
from sklearn.feature_selection import chi2

#standrenoX_train = renoX_train.apply(zscore, axis=1)
#standrenoX_test = renoX_test.apply(zscore, axis=1)

chi2, pval = chi2(renoX_train,y_train)
print(zip(renoX_train.columns,chi2))
print(zip(renoX_train.columns,pval))

[('OverallQual', 336.82882054406184), ('OverallCond', 134.59403496206971), ('FireplaceQu', 1422.6778226983854), ('GarageFinish', 396.10165960698356), ('GarageQual', 126.73613163553021), ('GarageCond', 123.53279955655236), ('PoolQC', 2506.5796703296764), ('ExterQual', 94.528256958824912), ('ExterCond', 21.513006256102322), ('BsmtQual', 206.57748623507032), ('BsmtCond', 85.888074704154945), ('BsmtFinType1', 909.39532669544042), ('Heatg_Floor', 1284.0000000000202), ('Heatg_GasA', 14.948885787701812), ('Heatg_GasW', 440.27700617283887), ('Heatg_Grav', 993.44444444446208), ('Heatg_OthW', 372.79166666666174), ('Heatg_Wall', 852.66666666665606), ('HeatingQC', 152.20188443995454), ('CentralAir', 50.201766542137463), ('Elec_FuseA', 557.92825091094085), ('Elec_FuseF', 516.08384615385125), ('Elec_FuseP', 653.77777777778863), ('Elec_Mix', 641.50000000001558), ('Elec_SBrkr', 56.873808936441677), ('KitchenQual', 118.73448403629172), ('Func_Maj1', 631.08933725004874), ('Func_Maj2', 513.28333333333717

In [129]:
model.fit(renoX_train, y_train)

Ridge(alpha=3, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [130]:
y_pred = model.predict(renoX_test)

cross_val_score(model, renoX_test, y=y_test)

array([ 0.67808771,  0.72797552,  0.78951519])

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. What property characteristics predict an "abnormal" sale?

---

The `SaleCondition` feature indicates the circumstances of the house sale. From the data file, we can see that the possibilities are:

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)
       
One of the executives at your company has an "in" with higher-ups at the major regional bank. His friends at the bank have made him a proposal: if he can reliably indicate what features, if any, predict "abnormal" sales (foreclosures, short sales, etc.), then in return the bank will give him first dibs on the pre-auction purchase of those properties (at a dirt-cheap price).

He has tasked you with determining (and adequately validating) which features of a property predict this type of sale. 

---

**Your task:**
1. Determine which features predict the `Abnorml` category in the `SaleCondition` feature.
- Justify your results.

This is a challenging task that tests your ability to perform classification analysis in the face of severe class imbalance. You may find that simply running a classifier on the full dataset to predict the category ends up useless: when there is bad class imbalance classifiers often tend to simply guess the majority class.

It is up to you to determine how you will tackle this problem. I recommend doing some research to find out how others have dealt with the problem in the past. Make sure to justify your solution. Don't worry about it being "the best" solution, but be rigorous.

Be sure to indicate which features are predictive (if any) and whether they are positive or negative predictors of abnormal sales.

In [ ]:
# A: